# CDR Analysis

This notebook contains exercises to analyze aggregated CDR data. It uses mock CDR data for Ghana from February to May 2020. 

# Environment set-up
Run the cell below:

In [ ]:
#------------------------------------------------------------------------
# Libraries installation

!pip install geopandas

#------------------------------------------------------------------------
# User defined functions

def time_complete(data, timefreq = 'D'):
    data = data.reset_index()
    timevar = data.columns[0]
    data[timevar] = data[timevar].astype('datetime64[D]')
    full_time_range = pd.date_range(data[timevar].min(),  
                                            data[timevar].max(), 
                                            freq = timefreq)
    data = data.set_index(timevar)
    data = data.reindex(full_time_range,  fill_value=0)
    data.index.name = 'date'
    return(data)

def day_lag(df):
    # Makse sure date is datetime type
    df['date'] = pd.to_datetime(df['date'])
    
    # Sort by region and date
    df = df.sort_values(['date'])
    
    # Lag value
    df['value_l'] = df['value'].shift(1)
    
    # Drop values if missing dates
    value_l = df['value_l'].where(df.date.diff() == dt.timedelta(days = 1), np.nan)
    
    return value_l


# Let's start

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import datetime as dt

First, let's import the datasets we will use on this exercise

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!ls

Before we start, let's load and have a look at each one of the indicators.

In [ ]:
# Load transactions per day data
trans = pd.read_csv('transactions_per_day.csv')
trans.head()

In [ ]:
# Load subscribers per day data
subs = pd.read_csv('subscribers_per_day.csv')
subs.head()

In [ ]:
# Load movements
mov = pd.read_csv('movements_per_day.csv')
mov.head()

## Exercise 1 - Quality checks

First step is to take a quick look at the completeness and consistency of the data.

- Check the number of regions per day
- Check time completeness of the series
- Compare number of subscribers to the number of calls


### Part 1 instructions:
1. Aggregate the number of transactions per date for `transaction data` and `subscriber data`. Store the result in a new DataFrame called `trans_day` and `subs_day`
    - Calculate the number of unique regions per day (TIP: use the `pd.Series.nunique` function.
    - Calculate sum of total transactions per day
2. Test if any date has fewer than 16 regions
3. Run the time complete function (user defined) for `subscriber data` to create a new DataFrame replacing missing rows with zeros:
    `trans_day_tcomplete = time_complete(subs_day)`
4. Plot the `value` column of the time complete data frame.

Do you see anything unusual?



In [ ]:
trans_day = trans.groupby('date').agg({'value': np.sum, 'pcod':pd.Series.nunique}).reset_index().sort_values('date')
trans_day

In [ ]:
lower_reg = unique_reg[unique_reg.pcod < 16]
lower_reg

In [ ]:
subs_day = subs.groupby('date').agg({'value': np.sum, 'pcod':pd.Series.nunique}).reset_index().sort_values('date')
subs_day

In [ ]:
# Note: the date index is not correct?
subs_day_tcomplete = time_complete(subs_day)
subs_day_tcomplete

In [ ]:
subs_day.plot(x = 'date',y = 'value')

## Exercise 2 - Changes over time

Now let's look how movement has changed over time. 

For simplicity we will use country level data and only look at movements between two different regions. Here's a quick summary of the comparisons we'll do:

- Absolute values
- Change from previous day
- Change from Baseline (defined as the average from February 1st to March 15th)

Also here is a time-line to help interpret the results:
- February 1st to March 15th: Baseline period
- March 16th: initial restrictions imposed
- March 30th: Lockdown measures on parts of Accra and Kumasi metropolitan areas

### Instructions
1. On the `mov` DataFrame, remove rows where users move within regions, that is `pcod_to` and `pcod_from` are the same
2. Aggregate `mov` DataFrame by `date` to have country level data
3. Use the `day_lag()` function to create the value from previous day for each date
4. Use the `bl_values()` function to calculate the baseline for each date
5. Create percent changes from previous day and from baseline columns.
6. Create 3 different line plots:
    - Level values of total movements
    - Percent change from previous day
    - Percent change from baseline

How does these 3 plots compare?

In [ ]:
# Q1
mov = mov[mov['pcod_from'] != mov['pcod_to']]
mov

In [ ]:
# Q2
mov_day = mov.groupby('date').agg({'value' : np.sum}).reset_index()
mov_day

In [ ]:
# Q3
mov_day['prev_value'] = day_lag(mov_day)
mov_day

In [ ]:
# Q4
mov_day = bl_values(mov_day)

In [ ]:
# Q5
mov_day['p_change_prev'] = (mov_day['value'] - mov_day['prev_value'])/mov_day['prev_value']
mov_day['p_change_bl'] = (mov_day['value'] - mov_day['value_bl'])/mov_day['value_bl']


In [ ]:
mov_day

In [ ]:
# Q6
mov_day.plot(x = 'date', y = 'value')

In [ ]:
mov_day.plot(x = 'date', y = 'p_change_prev')

In [ ]:
mov_day.plot(x = 'date', y = 'p_change_bl')

### Function definitions:

Before we move to part 4, lets review what the `bl_value()` function is doing.

In [ ]:
def bl_values(df):
    # Makse sure date is datetime type
    df['date'] = pd.to_datetime(df['date'])

    # Create weekday variable to calculate baseline values
    df['weekday'] = df['date'].dt.dayofweek

    # Keep only entries from Feb 1st to Mar 15th
    bl = df[df['date'] < dt.datetime(2020, 3, 16)]

    # Calculate baseline averages for each weekday
    # bl_averages = bl.groupby(['pcod', 'weekday']).agg({'value': np.mean}).reset_index()
    bl_averages = bl.groupby(['weekday']).agg({'value': np.mean}).reset_index()
    
    # Merge bl averages as a column on original df
    # ndf = df.merge(bl_averages, on = ['pcod', 'weekday'],
    #                suffixes = ('', '_bl')).drop('weekday', axis = 1)
    ndf = df.merge(bl_averages, on = ['weekday'],
                  suffixes = ('', '_bl')).drop('weekday', axis = 1)

    return ndf



## Exercise 3 - Choropleth

In [ ]:
import folium

In [ ]:
gdf = gpd.read_file("admin1.geojson")
a1_geo = r'admin1.geojson'

In [ ]:
gdf.plot()


In [ ]:
# subs_day = subs_day.reset_index()
subs['month'] = subs['date'].dt.month
subs_month = subs.groupby(['month', 'pcod']).agg({'value': np.sum}).reset_index()
cho_data4 = subs_month[subs_month['month'] == 4]

In [ ]:

m = folium.Map(location=[7.28, -0.97], zoom_start=7)

folium.Choropleth(
    geo_data=a1_geo,
    name="May",
    data=cho_data5,
    columns=["pcod", "value"],
    key_on="feature.properties.pcod",
    fill_color="BuPu",
    fill_opacity=0.7,
    line_opacity=0.2,
#     highlight = True,
#     reset = True,
#     overlay = False
).add_to(m)

# Add control to the side 
folium.LayerControl().add_to(m)

m